In [8]:
import pandas as pd
import json
from pydantic import BaseModel, TypeAdapter, RootModel
from pandas_to_pydantic_test.libraryTypes import Book, Author, Library
from pandas_to_pydantic_test.config import LIBRARY_DATA_PATH
from pandas_to_pydantic import expandAnnotation, getBaseFields, getListFields, serializeDataframe, dataframeToPydantic

In [9]:
data = pd.read_csv(LIBRARY_DATA_PATH)

In [24]:
class Book(BaseModel):
    BookID: int
    Title: str
    PublishedYear: int

class Author(BaseModel):
    AuthorName: str
    BookList: list[Book]

class Genre(BaseModel):
    Genre: str
    BookList: list[Book]

In [23]:
simpleData = data[['BookID','Title','AuthorName', 'Genre', 'PublishedYear']].drop_duplicates().sort_values("BookID")

In [29]:
simpleData

,BookID,Title,AuthorName,Genre,PublishedYear
0,1,Harry Potter and the Philosopher's Stone,J.K. Rowling,Fantasy,1997
1,2,Harry Potter and the Chamber of Secrets,J.K. Rowling,Fantasy,1998
3,3,1984,George Orwell,Dystopian Fiction,1949
9,4,Animal Farm,George Orwell,Political Satire,1945
5,5,Pride and Prejudice,Jane Austen,Romance,1813
6,7,Murder on the Orient Express,Agatha Christie,Mystery,1934
7,9,Adventures of Huckleberry Finn,Mark Twain,Adventure,1884
2,10,The Adventures of Tom Sawyer,Mark Twain,Adventure,1876
4,11,The Hobbit,J.R.R. Tolkien,Fantasy,1937
10,12,The Lord of the Rings,J.R.R. Tolkien,Fantasy,1954


In [28]:
dataframeToPydantic(simpleData, Author).model_dump()

[{'AuthorName': 'J.K. Rowling',
  'BookList': [{'BookID': 1,
    'Title': "Harry Potter and the Philosopher's Stone",
    'PublishedYear': 1997},
   {'BookID': 2,
    'Title': 'Harry Potter and the Chamber of Secrets',
    'PublishedYear': 1998}]},
 {'AuthorName': 'George Orwell',
  'BookList': [{'BookID': 3, 'Title': '1984', 'PublishedYear': 1949},
   {'BookID': 4, 'Title': 'Animal Farm', 'PublishedYear': 1945}]},
 {'AuthorName': 'Jane Austen',
  'BookList': [{'BookID': 5,
    'Title': 'Pride and Prejudice',
    'PublishedYear': 1813}]},
 {'AuthorName': 'Agatha Christie',
  'BookList': [{'BookID': 7,
    'Title': 'Murder on the Orient Express',
    'PublishedYear': 1934}]},
 {'AuthorName': 'Mark Twain',
  'BookList': [{'BookID': 9,
    'Title': 'Adventures of Huckleberry Finn',
    'PublishedYear': 1884},
   {'BookID': 10,
    'Title': 'The Adventures of Tom Sawyer',
    'PublishedYear': 1876}]},
 {'AuthorName': 'J.R.R. Tolkien',
  'BookList': [{'BookID': 11, 'Title': 'The Hobbit', 'P

In [ ]:
dataframeToPydantic(simpleData, Genre).model_dump()

In [3]:
libraryListRoot = dataframeToPydantic(data, Library)

In [4]:
libraryListRoot.root

[Library(LibraryID=1, LibraryName='City Central Library', Location='Cityville', EstablishedYear=1950, BookCollectionSize=50000, AuthorList=[Author(AuthorID=1, AuthorName='J.K. Rowling', AuthorBirthdate='1965-07-31', BookList=[Book(BookID=1, Title="Harry Potter and the Philosopher's Stone", Genre='Fantasy', PublishedYear=1997, AvailableCopies=5), Book(BookID=2, Title='Harry Potter and the Chamber of Secrets', Genre='Fantasy', PublishedYear=1998, AvailableCopies=3)]), Author(AuthorID=5, AuthorName='Mark Twain', AuthorBirthdate='1835-11-30', BookList=[Book(BookID=10, Title='The Adventures of Tom Sawyer', Genre='Adventure', PublishedYear=1876, AvailableCopies=2)])]),
 Library(LibraryID=2, LibraryName='Greenwood Public Library', Location='Greenwood', EstablishedYear=1975, BookCollectionSize=35000, AuthorList=[Author(AuthorID=2, AuthorName='George Orwell', AuthorBirthdate='1903-06-25', BookList=[Book(BookID=3, Title='1984', Genre='Dystopian Fiction', PublishedYear=1949, AvailableCopies=7)]),

In [5]:
libraryListRoot.model_dump()

[{'LibraryID': 1,
  'LibraryName': 'City Central Library',
  'Location': 'Cityville',
  'EstablishedYear': 1950,
  'BookCollectionSize': 50000,
  'AuthorList': [{'AuthorID': 1,
    'AuthorName': 'J.K. Rowling',
    'AuthorBirthdate': '1965-07-31',
    'BookList': [{'BookID': 1,
      'Title': "Harry Potter and the Philosopher's Stone",
      'Genre': 'Fantasy',
      'PublishedYear': 1997,
      'AvailableCopies': 5},
     {'BookID': 2,
      'Title': 'Harry Potter and the Chamber of Secrets',
      'Genre': 'Fantasy',
      'PublishedYear': 1998,
      'AvailableCopies': 3}]},
   {'AuthorID': 5,
    'AuthorName': 'Mark Twain',
    'AuthorBirthdate': '1835-11-30',
    'BookList': [{'BookID': 10,
      'Title': 'The Adventures of Tom Sawyer',
      'Genre': 'Adventure',
      'PublishedYear': 1876,
      'AvailableCopies': 2}]}]},
 {'LibraryID': 2,
  'LibraryName': 'Greenwood Public Library',
  'Location': 'Greenwood',
  'EstablishedYear': 1975,
  'BookCollectionSize': 35000,
  'AuthorL